# *Algo 3.1 for the top down model*

In [2]:
import logging
import numpy as np
from scipy.special import hyp1f1, iv
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    # Numerical methods and tolerances
    # Constants for numerical stability and convergence
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    # Maximum iterations for numerical methods
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    # Number of grid points for numerical integration and interpolation
    grid_points: int = 500
    # Tolerance for numerical methods
    tolerance_scale: float = 1e-6

config = SimulationConfig()

In [3]:
def validate_parameters(model_params: ModelParams) -> bool:
    """Validate CIR parameters and check Feller condition"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"] 
        sigma = model_params["sigma"]
        
        assert kappa > 0, "kappa must be positive"
        assert theta > 0, "theta must be positive" 
        assert sigma > 0, "sigma must be positive"
        
        feller_condition = 2 * kappa * theta >= sigma**2
        if not feller_condition:
            logger.warning(f"Feller condition violated: 2kappa*theta={2*kappa*theta:.6f} < sigma^2={sigma**2:.6f}")
            
        logger.info(f"Parameters: κ={kappa:.6f}, θ={theta:.6f}, σ={sigma:.6f}")
        logger.info(f"Feller condition {'satisfied' if feller_condition else 'violated'}")
        
        return feller_condition
        
    except Exception as e:
        logger.error(f"Parameter validation failed: {e}")
        raise

In [4]:
def _hyp1f1(a: float, b: float, z: float, method: str = 'primary') -> float:
    """Hypergeometric function evaluation with fallbacks"""
    try:
        # Primary method
        if method == 'primary' and abs(z) < 100 and abs(a) < 50:
            result = hyp1f1(a, b, z)
            if np.isfinite(result) and abs(result) < 1e10:
                return result
        
        # # Fallback for large arguments using series truncation
        # if abs(z) > 100 or abs(a) > 50:
        #     logger.debug(f"Using series approximation for hyp1f1({a:.3f}, {b:.3f}, {z:.3f})")
        #     # For large z, use asymptotic expansion if a is negative integer-like
        #     if a < 0 and abs(a - round(a)) < config.eps_small:
        #         n = int(abs(round(a)))
        #         if n < 20:  # Finite series
        #             result = sum((a + k) * z**k / math.factorial(k) / (b + k) 
        #                        for k in range(n + 1))
        #             return result
        
        # Fallback resort: clip extreme values
        a_clip = np.clip(a, -50, 50)
        z_clip = np.clip(z, -100, 100)
        result = hyp1f1(a_clip, b, z_clip)
        
        if not np.isfinite(result):
            logger.warning(f"hyp1f1 returned non-finite: a={a}, b={b}, z={z}")
            return 1.0  
            
        return result
        
    except Exception as e:
        logger.warning(f"hyp1f1 evaluation failed: {e}, using fallback")
        return 1.0

In [5]:
def hyp1f1_derivative_a(a: float, b: float, z: float, delta: float = None) -> float:
    """Numerical derivative of hypergeometric function"""
    if delta is None:
        delta = max(config.eps_small, abs(a) * config.tolerance_scale)
    
    try:
        f_plus = _hyp1f1(a + delta, b, z)
        f_minus = _hyp1f1(a - delta, b, z)
        derivative = (f_plus - f_minus) / (2 * delta)
        
        if not np.isfinite(derivative):
            # Try smaller delta
            delta *= 0.1
            f_plus = _hyp1f1(a + delta, b, z)
            f_minus = _hyp1f1(a - delta, b, z)
            derivative = (f_plus - f_minus) / (2 * delta)
            
        return derivative if np.isfinite(derivative) else 0.0
        
    except Exception as e:
        logger.warning(f"Derivative computation failed: {e}")
        return 0.0

In [6]:
def laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        logger.debug(f"Laplace transform: a={a:.6f}, b={b:.6f}, z1={z1:.6f}, z2={z2:.6f}")
        
        numerator = _hyp1f1(a, b, z1)
        denominator = _hyp1f1(a, b, z2)
        
        if abs(denominator) < config.eps_zero:
            logger.warning("Near-zero denominator in Laplace transform")
            return 0.99  # Safe fallback
            
        result = numerator / denominator
        
        # Stability check
        if not (0 <= result <= 1):
            logger.warning(f"Laplace transform out of bounds: {result:.6f}")
            result = np.clip(result, config.eps_zero, 1 - config.eps_zero)
            
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5  # Conservative fallback

In [7]:
def compute_coefficients(y: float, H: float, model_params: ModelParams, n_roots: int = 15) -> Tuple[List[float], List[float]]:
    """Compute eigenfunction expansion coefficients as given by eqn 20 in section 4.2. root finding given in appendix B of the paper"""
    start_time = time.time()
    
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        c_bar = 2 * kappa * theta / sigma**2
        H_bar = 2 * kappa * H / sigma**2
        y_bar = 2 * kappa * y / sigma**2

        if c_bar < 1 or H_bar < c_bar:
            logger.error(f"Invalid parameters: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
            return [], []
        
        logger.debug(f"Computing coefficients: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
        
        def objective(alpha):
            return _hyp1f1(alpha, c_bar, H_bar)
        
        roots = []
        n = 0
        alpha_prev = 0

        # Find the first root in (-c_bar / H_bar, 0)
        left = -c_bar / H_bar
        right = 0
        try:
            alpha_curr = brentq(objective, left, right)
            roots.append(alpha_curr)
        except Exception as e:
            logger.error(f"Failed to find first root: {e}")
            return [], []

        # Find subsequent roots
        while len(roots) < n_roots:
            n += 1
            # Predict the next root using extrapolation
            alpha_hat = 2 * alpha_curr - alpha_prev

            # Inner loop to bracket the root
            bracket_found = False
            for attempt in range(20):  
                try:
                    test_val = objective(alpha_hat - 1)
                    if np.sign(test_val) == (-1)**n:
                        alpha_hat = alpha_hat - 1
                    else:
                        bracket_found = True
                        break
                except:
                    alpha_hat = alpha_hat - 1
                    if attempt > 10:  # Give up after many attempts
                        break
            
            if not bracket_found:
                logger.warning(f"Could not bracket root {n+1}, stopping at {len(roots)} roots")
                break
                
            try:
                alpha_next = brentq(objective, alpha_hat - 1, alpha_hat)
                roots.append(alpha_next)
                alpha_prev = alpha_curr
                alpha_curr = alpha_next
            except Exception as e:
                logger.warning(f"Failed to find root {n+1}: {e}, stopping at {len(roots)} roots")
                break
        
        if len(roots) < 5:
            logger.error(f"Insufficient roots found: {len(roots)}")
            return [], []
        
        # Compute coefficients
        eta_n, beta_n = [], []
        for alpha in roots:
            try:

                if abs(alpha) <= config.eps_zero:
                    logger.warning(f"Skipping near-zero alpha={alpha:.6f}")
                    eta_n.append(0.0)
                    beta_n.append(0.0)
                    continue
                
                eta = -kappa * alpha
                eta_n.append(eta)
                
                numerator = _hyp1f1(alpha, c_bar, y_bar)
                derivative = hyp1f1_derivative_a(alpha, c_bar, H_bar)
                
                if abs(derivative) < config.eps_zero:
                    logger.warning(f"Near-zero derivative for alpha={alpha:.6f}")
                    beta = 0.0
                else:
                    beta = -numerator / (alpha * derivative)

                if abs(alpha*derivative) < config.eps_small:
                    logger.warning(f"Near-zero product of alpha and derivative for alpha={alpha:.6f}")
                    beta = 0.0
                elif not np.isfinite(beta):
                    logger.warning(f"Non-finite beta for alpha={alpha:.6f}, setting to 0.0")
                    beta = 0.0
                else:
                    beta = -numerator / (alpha * derivative)
                    
                beta_n.append(beta)
                
            except Exception as e:
                logger.warning(f"Coefficient computation failed for alpha={alpha:.6f}: {e}")
                eta_n.append(0.0)
                beta_n.append(0.0)
        
        # Filter out problematic coefficients
        valid_indices = [i for i, (eta, beta) in enumerate(zip(eta_n, beta_n)) 
                        if np.isfinite(eta) and np.isfinite(beta)]
        
        eta_n = [eta_n[i] for i in valid_indices]
        beta_n = [beta_n[i] for i in valid_indices]
        
        logger.debug(f"Computed {len(eta_n)} valid coefficients in {time.time() - start_time:.3f}s")
        
        return eta_n, beta_n
        
    except Exception as e:
        logger.error(f"Coefficient computation failed: {e}")
        return [], []

In [8]:
def compute_survival_probability_p1(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Compute survival probability"""
    try:
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients available, using fallback")
            return 0.5
        
        p1 = sum(beta * np.exp(-eta * tau) for eta, beta in zip(eta_n, beta_n) 
                if np.isfinite(beta) and np.isfinite(eta))
        
        # Stability bounds
        p1 = np.clip(p1, config.eps_zero, 1 - config.eps_zero)
        
        logger.debug(f"Survival probability: {p1:.6f}")
        return p1
        
    except Exception as e:
        logger.error(f"Survival probability computation failed: {e}")
        return 0.5

In [9]:
def cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            return 0.0
            
        q = 2 * kappa * theta / sigma**2 - 1
        exp_term = np.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        # Compute in log space for stability
        log_factor = np.log(a) - (ax + b * y)
        log_power = (q / 2) * (np.log(ax) - np.log(by))
        
        # Bessel function argument
        bessel_arg = 2 * np.sqrt(a * b * x * y)
        if bessel_arg > 100:  # Use asymptotic approximation
            log_bessel = bessel_arg - 0.5 * np.log(2 * np.pi * bessel_arg)
        else:
            bessel_val = iv(q, bessel_arg)
            if bessel_val <= 0:
                return 0.0
            log_bessel = np.log(bessel_val)
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  # Prevent overflow
            return 0.0
            
        return np.exp(log_density)
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

In [10]:
def select_dominating_intensity_H(y: float, model_params: ModelParams, 
                              t_C: float = 1.0, t_eta: float = 1.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, bounds=(H_min, H_max), method='bounded')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)

        H_ret = min(H_final, theta*8)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_ret
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)


In [11]:
def sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)  # Reasonable upper bound
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        # Compute transition density
        g_vals = np.array([cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        # Compute convolution integral (simplified for robustness)
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        
        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = compute_survival_probability_p1(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)  # Avoid negative densities
        f_vals /= np.sum(f_vals)  # Normalize

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        

        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        # cdf_vals = np.cumsum((f_vals[:-1] + f_vals[1:]) * 0.5 * np.diff(x_grid))
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            # Uniform fallback
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        # Ensure monotonicity
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])
        
        # Sample
        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            # Linear interpolation fallback
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        # Final bounds check
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)


In [12]:
def sample_hitting_time_from_v(nu_t: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample hitting time"""
    try:
        eta_n, beta_n = compute_coefficients(nu_t, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients for hitting time, using uniform")
            return np.random.uniform(0, tau)
        
        P = 1 - compute_survival_probability_p1(nu_t, H, tau, model_params)
        P = max(P, config.eps_zero)
        
        U = np.random.uniform(0, P)
        
        def objective(s):
            integral = sum(beta * (1 - np.exp(-eta * s)) for beta, eta in zip(beta_n, eta_n)
                          if eta > config.eps_zero)
            return integral - U

        try:
            result = brentq(objective, 0, tau)
            return min(result, tau)
        except:
            logger.warning("Hitting time root finding failed, using approximation")
            return np.random.uniform(0, tau * 0.8)
            
    except Exception as e:
        logger.error(f"Hitting time sampling failed: {e}")
        return np.random.uniform(0, tau * 0.5)

In [13]:
def sample_loss() -> float:
    """Sample loss from a uniform distribution as per the original model"""
    return np.random.uniform(0.24, 0.96)

In [14]:
def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step in the top-down model.

    Parameters:
        t_prev: float
            Previous default time T_{n-1}
        lambda_prev: float
            Intensity after last jump, λ_{T_{n-1}} 
        T_max: float
            Final time horizon for simulation
        model_params: dict
            Contains CIR and model parameters: kappa, theta, sigma, gamma

    Returns:
        Optional[Tuple[float, float]]:
            If a default occurs before T_max:
                (T_n, ν_{T_n})
            Else:
                None (no more defaults)
    """
    try:
        # STEP 1: Initialize y := λ_{T_{n-1}}, t := T_{n-1}
        y = max(lambda_prev, config.eps_small) # Ensure y is positive
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            # STEP 2: Select H>y
            H = select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            # STEP 3: Sample τ ∼ Exp(H), propose next interarrival time
            tau = np.random.exponential(scale = 1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            # STEP 4: Compute survival probability p1 = P(ν stays below H up to τ)
            p1 = compute_survival_probability_p1(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            # STEP 5: First rejection test with u1 ∼ Unif(0,1)
            u1 = np.random.uniform()
            
            if u1 > p1:
                # Process hits H → sample σ_H from hitting time density
                logger.info(f"Process hits boundary H={H:.6f} at t={t:.6f}, sampling hitting time")
                sigma_H = sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                    
                y = H  # Update intensity and repeat
                continue
            else:
                # Process survives to τ
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                
                # STEP 6: Sample ν_{T_n} from conditional density f(.; ν_t, H, τ)
                nu_Tn = sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                # STEP 7: Second rejection test using u2 ∼ Unif(0,1)
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    # Accept default at T_n = t
                    # ell_n = sample_loss()  # Draw loss ℓ_n ∼ loss_distribution()
                    
                    # logger.debug(f"Default at t={t:.6f}, ν={nu_Tn:.6f}, loss={ell_n:.6f}")
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    # Reject - update ν_t and go back to step 2
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function that generates a sequence of default times and losses.
    
    Parameters:
        lambda_0: float
            Initial intensity λ_0
        T_max: float
            Time horizon
        model_params: ModelParams
            Dictionary containing kappa, theta, sigma, gamma
        max_defaults: int
            Maximum number of defaults to simulate
            
    Returns:
        List[Tuple[float, float]]: List of (default_time, loss) pairs
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")
    
    # Validate parameters
    if not validate_parameters(model_params):
        logger.warning("Proceeding despite Feller condition violation")
    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    # gamma = model_params.get("gamma", 2.99)  # Default jump size
    gamma = model_params["gamma"]
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n  # Update intensity at default with jump
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults

In [24]:
# paper parameters for testing
model_params = {
    "kappa": 2.62,    # Mean reversion speed
    "theta": 1.61,    # Long-term mean
    "sigma": 0.62,    # Volatility
    "gamma": 0.99     # Jump size multiplier
}

defaults = simulate_defaults(
    lambda_0=0.7,
    T_max=5,
    model_params=model_params,
    max_defaults=100
)

print(f"Generated {len(defaults)} defaults:")
for i, (t, lam, loss) in enumerate(defaults): 
    print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")


2025-06-03 23:09:45,123 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-03 23:09:45,124 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-06-03 23:09:45,125 - INFO - Feller condition satisfied
2025-06-03 23:09:45,126 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-06-03 23:09:45,127 - INFO - Computed survival probability p1=0.995587 for y=0.700000, H=2.295761, τ=0.436066
2025-06-03 23:09:45,128 - INFO - Process survives to t=0.436066, sampling conditional intensity
2025-06-03 23:09:46,047 - INFO - Iteration 2: Selected H=2.378413 for y=1.060000
2025-06-03 23:09:46,048 - INFO - Computed survival probability p1=0.999229 for y=1.060000, H=2.378413, τ=0.250212
2025-06-03 23:09:46,048 - INFO - Process survives to t=0.686278, sampling conditional intensity
2025-06-03 23:09:46,923 - INFO - Iteration 3: Selected H=2.549183 for y=1.560000
2025-06-03 23:09:46,924 - INFO - Computed survival probability p1=0.999146 for y=1.560000, H=2.549183, τ=0.148

Generated 4 defaults:
  1: t=0.835104, itensity=1.946294, loss=0.329590
  2: t=2.272151, itensity=2.690892, loss=0.536254
  3: t=4.694635, itensity=3.022874, loss=0.709973
  4: t=4.704315, itensity=3.242730, loss=0.344214


In [16]:
L = sum(loss for _, _, loss in defaults)
print(f"Total loss: {L:.6f}")

Total loss: 6.621960


In [15]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}

L_5 = []

for _ in range(10):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 10 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")



2025-06-03 23:21:23,101 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-03 23:21:23,104 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-06-03 23:21:23,105 - INFO - Feller condition satisfied
2025-06-03 23:21:23,105 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-06-03 23:21:23,107 - INFO - Computed survival probability p1=0.758375 for y=0.700000, H=2.295761, τ=1.585343
2025-06-03 23:21:23,108 - INFO - Process survives to t=1.585343, sampling conditional intensity
2025-06-03 23:21:24,065 - INFO - Iteration 2: Selected H=2.373130 for y=1.040000
2025-06-03 23:21:24,066 - INFO - Computed survival probability p1=1.000000 for y=1.040000, H=2.373130, τ=0.102013
2025-06-03 23:21:24,066 - INFO - Process survives to t=1.687356, sampling conditional intensity
2025-06-03 23:21:24,928 - INFO - Iteration 3: Selected H=2.378413 for y=1.060000
2025-06-03 23:21:24,929 - INFO - Computed survival probability p1=0.997963 for y=1.060000, H=2.378413, τ=0.000

Generated 15 defaults:
  1: t=1.688063, itensity=1.995140, loss=0.924384
  2: t=1.872430, itensity=2.954475, loss=0.923712
  3: t=1.899621, itensity=3.485241, loss=0.625656
  4: t=2.039885, itensity=3.788709, loss=0.905010
  5: t=2.050379, itensity=4.228297, loss=0.558837
  6: t=2.074074, itensity=5.057354, loss=0.922851
  7: t=2.694412, itensity=2.723366, loss=0.597791
  8: t=3.180866, itensity=2.906210, loss=0.844901
  9: t=3.215884, itensity=3.402281, loss=0.383659
  10: t=3.588237, itensity=3.176201, loss=0.424599
  11: t=3.792856, itensity=2.840088, loss=0.302148
  12: t=3.793576, itensity=3.244431, loss=0.322364
  13: t=3.964001, itensity=3.411510, loss=0.365399
  14: t=4.086125, itensity=3.965533, loss=0.387320
  15: t=4.372004, itensity=3.017663, loss=0.284288
Total loss: 8.772921


2025-06-03 23:21:40,442 - INFO - Default accepted at t=0.490315, ν=1.240000.
2025-06-03 23:21:40,442 - INFO - Iteration 1: Selected H=2.579349 for y=1.627448
2025-06-03 23:21:40,443 - INFO - Computed survival probability p1=0.999880 for y=1.627448, H=2.579349, τ=0.095534
2025-06-03 23:21:40,443 - INFO - Process survives to t=0.585849, sampling conditional intensity
2025-06-03 23:21:41,298 - INFO - Default accepted at t=0.585849, ν=1.240000.
2025-06-03 23:21:41,299 - INFO - Iteration 1: Selected H=2.550573 for y=1.563207
2025-06-03 23:21:41,300 - INFO - Computed survival probability p1=1.000000 for y=1.563207, H=2.550573, τ=0.045934
2025-06-03 23:21:41,300 - INFO - Process survives to t=0.631783, sampling conditional intensity
2025-06-03 23:21:42,151 - INFO - Default accepted at t=0.631783, ν=1.600000.
2025-06-03 23:21:42,152 - INFO - Iteration 1: Selected H=3.013893 for y=2.325181
2025-06-03 23:21:42,153 - INFO - Computed survival probability p1=0.971240 for y=2.325181, H=3.013893, τ=0

Generated 9 defaults:
  1: t=0.490315, itensity=1.627448, loss=0.391362
  2: t=0.585849, itensity=1.563207, loss=0.326472
  3: t=0.631783, itensity=2.325181, loss=0.732506
  4: t=1.526815, itensity=1.947574, loss=0.441318
  5: t=2.154050, itensity=1.973856, loss=0.317026
  6: t=3.204067, itensity=2.592106, loss=0.499905
  7: t=3.288774, itensity=3.506048, loss=0.896990
  8: t=3.401348, itensity=4.086340, loss=0.550739
  9: t=3.773284, itensity=3.200774, loss=0.880364
Total loss: 5.036682


2025-06-03 23:21:52,631 - INFO - Default accepted at t=0.280136, ν=1.280000.
2025-06-03 23:21:52,631 - INFO - Iteration 1: Selected H=2.827508 for y=2.066518
2025-06-03 23:21:52,632 - INFO - Computed survival probability p1=1.000000 for y=2.066518, H=2.827508, τ=0.016683
2025-06-03 23:21:52,632 - INFO - Process survives to t=0.296819, sampling conditional intensity
2025-06-03 23:21:53,483 - INFO - Default accepted at t=0.296819, ν=2.045853.
2025-06-03 23:21:53,484 - INFO - Iteration 1: Selected H=3.283020 for y=2.655199
2025-06-03 23:21:53,485 - INFO - Computed survival probability p1=0.998803 for y=2.655199, H=3.283020, τ=0.029886
2025-06-03 23:21:53,485 - INFO - Process survives to t=0.326704, sampling conditional intensity
2025-06-03 23:21:54,336 - INFO - Default accepted at t=0.326704, ν=2.841063.
2025-06-03 23:21:54,336 - INFO - Iteration 1: Selected H=3.883777 for y=3.321164
2025-06-03 23:21:54,337 - INFO - Computed survival probability p1=0.984277 for y=3.321164, H=3.883777, τ=0

...

Generated 16 defaults:
  1: t=0.484003, itensity=1.385562, loss=0.409659
  2: t=1.987113, itensity=2.371637, loss=0.678421
  3: t=2.320318, itensity=2.190738, loss=0.320348
  4: t=2.842815, itensity=2.881663, loss=0.259978
  5: t=2.869056, itensity=3.524761, loss=0.416733
  6: t=3.222407, itensity=3.637117, loss=0.932375
  7: t=3.298112, itensity=3.744535, loss=0.769797
  8: t=3.323817, itensity=4.193905, loss=0.794322
  9: t=3.469883, itensity=4.298529, loss=0.952934
  10: t=3.489887, itensity=4.701277, loss=0.580494
  11: t=3.647844, itensity=4.098992, loss=0.768773
  12: t=3.744275, itensity=4.266028, loss=0.665571
  13: t=4.124965, itensity=3.293548, loss=0.348353
  14: t=4.293339, itensity=3.447727, loss=0.821100
  15: t=4.576261, itensity=3.631435, loss=0.951725
  16: t=4.859724, itensity=2.969683, loss=0.652086
Total loss: 10.322667


2025-06-03 23:23:34,132 - INFO - Iteration 2: Selected H=2.287951 for y=0.660000
2025-06-03 23:23:34,133 - INFO - Computed survival probability p1=0.999999 for y=0.660000, H=2.287951, τ=0.109228
2025-06-03 23:23:34,133 - INFO - Process survives to t=0.123896, sampling conditional intensity
2025-06-03 23:23:34,990 - INFO - Default accepted at t=0.123896, ν=0.980000.
2025-06-03 23:23:34,990 - INFO - Iteration 1: Selected H=2.433763 for y=1.249993
2025-06-03 23:23:34,991 - INFO - Computed survival probability p1=0.992444 for y=1.249993, H=2.433763, τ=0.350191
2025-06-03 23:23:34,991 - INFO - Process survives to t=0.474087, sampling conditional intensity
2025-06-03 23:23:35,884 - INFO - Iteration 2: Selected H=2.500248 for y=1.440000
2025-06-03 23:23:35,885 - INFO - Computed survival probability p1=0.963043 for y=1.440000, H=2.500248, τ=0.588678
2025-06-03 23:23:35,886 - INFO - Process survives to t=1.062765, sampling conditional intensity
2025-06-03 23:23:36,820 - INFO - Default accepted 

Generated 12 defaults:
  1: t=0.123896, itensity=1.249993, loss=0.272720
  2: t=1.062765, itensity=1.727261, loss=0.249758
  3: t=1.448319, itensity=2.914717, loss=0.863351
  4: t=1.474729, itensity=3.180232, loss=0.503730
  5: t=1.489728, itensity=3.935734, loss=0.634639
  6: t=1.533794, itensity=4.348444, loss=0.695163
  7: t=1.592715, itensity=4.352956, loss=0.707336
  8: t=1.975412, itensity=3.365895, loss=0.541892
  9: t=2.130803, itensity=3.252161, loss=0.871087
  10: t=3.414545, itensity=1.747530, loss=0.518191
  11: t=4.474459, itensity=1.905692, loss=0.369386
  12: t=4.553232, itensity=1.947690, loss=0.290596
Total loss: 6.517850


2025-06-03 23:23:49,930 - INFO - Default accepted at t=0.350050, ν=1.200000.
2025-06-03 23:23:49,931 - INFO - Iteration 1: Selected H=2.736656 for y=1.923499
2025-06-03 23:23:49,932 - INFO - Computed survival probability p1=0.968333 for y=1.923499, H=2.736656, τ=0.479373
2025-06-03 23:23:49,932 - INFO - Process survives to t=0.829423, sampling conditional intensity
2025-06-03 23:23:50,847 - INFO - Default accepted at t=0.829423, ν=1.240000.
2025-06-03 23:23:50,848 - INFO - Iteration 1: Selected H=2.895189 for y=2.164764
2025-06-03 23:23:50,849 - INFO - Computed survival probability p1=0.988847 for y=2.164764, H=2.895189, τ=0.192339
2025-06-03 23:23:50,849 - INFO - Process survives to t=1.021762, sampling conditional intensity
2025-06-03 23:23:51,714 - INFO - Default accepted at t=1.021762, ν=2.121469.
2025-06-03 23:23:51,714 - INFO - Iteration 1: Selected H=3.128122 for y=2.469583
2025-06-03 23:23:51,715 - INFO - Computed survival probability p1=0.998839 for y=2.469583, H=3.128122, τ=0

Generated 10 defaults:
  1: t=0.350050, itensity=1.923499, loss=0.730807
  2: t=0.829423, itensity=2.164764, loss=0.934105
  3: t=1.021762, itensity=2.469583, loss=0.351631
  4: t=1.045533, itensity=3.169619, loss=0.806888
  5: t=1.321577, itensity=3.634145, loss=0.789382
  6: t=1.329787, itensity=4.555168, loss=0.856909
  7: t=2.107813, itensity=2.328013, loss=0.498252
  8: t=2.906324, itensity=1.741454, loss=0.395158
  9: t=3.498528, itensity=2.617176, loss=0.461794
  10: t=4.413164, itensity=2.065317, loss=0.579320
Total loss: 6.404245
Average total loss over 10 runs: 7.967507 ± 1.834272


In [16]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 2.99     # Jump size multiplier 
}

L_5 = []

for _ in range(5):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=10
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 5 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")



2025-06-03 23:24:05,048 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-03 23:24:05,050 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-06-03 23:24:05,051 - INFO - Feller condition satisfied
2025-06-03 23:24:05,052 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-06-03 23:24:05,054 - INFO - Computed survival probability p1=0.825018 for y=0.700000, H=2.295761, τ=1.304939
2025-06-03 23:24:05,054 - INFO - Process survives to t=1.304939, sampling conditional intensity
2025-06-03 23:24:06,036 - INFO - Default accepted at t=1.304939, ν=1.860000.
2025-06-03 23:24:06,037 - INFO - Iteration 1: Selected H=4.744567 for y=4.220429
2025-06-03 23:24:06,038 - INFO - Computed survival probability p1=0.990005 for y=4.220429, H=4.744567, τ=0.143583
2025-06-03 23:24:06,038 - INFO - Process survives to t=1.448522, sampling conditional intensity
2025-06-03 23:24:06,895 - INFO - Default accepted at t=1.448522, ν=3.334139.
2025-06-03 23:24:06,895 - INFO - Iter

Generated 2 defaults:
  1: t=1.304939, itensity=4.220429, loss=0.789441
  2: t=1.448522, itensity=5.488847, loss=0.720638
Total loss: 1.510079


2025-06-03 23:24:08,043 - INFO - Iteration 2: Selected H=2.299755 for y=0.720000
2025-06-03 23:24:08,043 - INFO - Computed survival probability p1=0.986679 for y=0.720000, H=2.299755, τ=0.020568
2025-06-03 23:24:08,044 - INFO - Process survives to t=0.047709, sampling conditional intensity
2025-06-03 23:24:08,911 - INFO - Iteration 3: Selected H=2.299755 for y=0.720000
2025-06-03 23:24:08,912 - INFO - Computed survival probability p1=0.993730 for y=0.720000, H=2.299755, τ=0.460686
2025-06-03 23:24:08,912 - INFO - Process survives to t=0.508395, sampling conditional intensity
2025-06-03 23:24:09,807 - INFO - Iteration 4: Selected H=2.299755 for y=0.720000
2025-06-03 23:24:09,807 - INFO - Computed survival probability p1=0.999997 for y=0.720000, H=2.299755, τ=0.097713
2025-06-03 23:24:09,808 - INFO - Process survives to t=0.606107, sampling conditional intensity
2025-06-03 23:24:10,656 - INFO - Iteration 5: Selected H=2.443454 for y=1.280000
2025-06-03 23:24:10,657 - INFO - Computed surv

Generated 3 defaults:
  1: t=0.999609, itensity=3.138767, loss=0.588216
  2: t=1.613095, itensity=3.862367, loss=0.546436
  3: t=1.626306, itensity=5.885651, loss=0.702519
Total loss: 1.837171


2025-06-03 23:24:14,494 - INFO - Default accepted at t=0.316154, ν=1.580000.
2025-06-03 23:24:14,495 - INFO - Iteration 1: Selected H=4.338385 for y=3.800249
2025-06-03 23:24:14,495 - INFO - Computed survival probability p1=0.995005 for y=3.800249, H=4.338385, τ=0.062022
2025-06-03 23:24:14,496 - INFO - Process survives to t=0.378177, sampling conditional intensity
2025-06-03 23:24:15,369 - INFO - Default accepted at t=0.378177, ν=3.116204.
2025-06-03 23:24:15,370 - INFO - Iteration 1: Selected H=5.068659 for y=4.552508
2025-06-03 23:24:15,371 - INFO - Computed survival probability p1=0.990215 for y=4.552508, H=5.068659, τ=0.258462
2025-06-03 23:24:15,371 - INFO - Process survives to t=0.636639, sampling conditional intensity
2025-06-03 23:24:16,249 - INFO - Default accepted at t=0.636639, ν=3.778582.
2025-06-03 23:24:16,250 - INFO - Iteration 1: Selected H=5.896754 for y=5.393471
2025-06-03 23:24:16,251 - WARNING - Skipping near-zero alpha=0.000000
2025-06-03 23:24:16,251 - INFO - Com

Generated 3 defaults:
  1: t=0.316154, itensity=3.800249, loss=0.742558
  2: t=0.378177, itensity=4.552508, loss=0.480369
  3: t=0.636639, itensity=5.393471, loss=0.540097
Total loss: 1.763024


2025-06-03 23:24:17,465 - INFO - Default accepted at t=0.203533, ν=1.920000.
2025-06-03 23:24:17,466 - INFO - Iteration 1: Selected H=4.040603 for y=3.487873
2025-06-03 23:24:17,466 - INFO - Computed survival probability p1=0.994399 for y=3.487873, H=4.040603, τ=0.070343
2025-06-03 23:24:17,467 - INFO - Process survives to t=0.273876, sampling conditional intensity
2025-06-03 23:24:18,348 - INFO - Default accepted at t=0.273876, ν=3.348358.
2025-06-03 23:24:18,349 - INFO - Iteration 1: Selected H=5.727624 for y=5.222350
2025-06-03 23:24:18,350 - WARNING - Skipping near-zero alpha=-0.000000
2025-06-03 23:24:18,350 - INFO - Computed survival probability p1=0.000020 for y=5.222350, H=5.727624, τ=0.235051
2025-06-03 23:24:18,351 - INFO - Process hits boundary H=5.727624 at t=0.273876, sampling hitting time
2025-06-03 23:24:18,351 - WARNING - Skipping near-zero alpha=-0.000000
2025-06-03 23:24:18,352 - WARNING - Skipping near-zero alpha=-0.000000
2025-06-03 23:24:18,352 - WARNING - Hitting 

Generated 2 defaults:
  1: t=0.203533, itensity=3.487873, loss=0.524372
  2: t=0.273876, itensity=5.222350, loss=0.626753
Total loss: 1.151126


2025-06-03 23:24:19,524 - INFO - Iteration 2: Selected H=2.276650 for y=0.600000
2025-06-03 23:24:19,525 - INFO - Computed survival probability p1=0.999413 for y=0.600000, H=2.276650, τ=0.343706
2025-06-03 23:24:19,525 - INFO - Process survives to t=0.490127, sampling conditional intensity
2025-06-03 23:24:20,425 - INFO - Iteration 3: Selected H=2.450072 for y=1.300000
2025-06-03 23:24:20,426 - INFO - Computed survival probability p1=0.999991 for y=1.300000, H=2.450072, τ=0.105050
2025-06-03 23:24:20,426 - INFO - Process survives to t=0.595177, sampling conditional intensity
2025-06-03 23:24:21,293 - INFO - Default accepted at t=0.595177, ν=1.500000.
2025-06-03 23:24:21,293 - INFO - Iteration 1: Selected H=3.701377 for y=3.124574
2025-06-03 23:24:21,294 - INFO - Computed survival probability p1=0.982231 for y=3.124574, H=3.701377, τ=0.356431
2025-06-03 23:24:21,294 - INFO - Process survives to t=0.951608, sampling conditional intensity
2025-06-03 23:24:22,158 - INFO - Default accepted 

Generated 7 defaults:
  1: t=0.595177, itensity=3.124574, loss=0.543336
  2: t=0.951608, itensity=3.909077, loss=0.544528
  3: t=2.262736, itensity=3.309142, loss=0.342058
  4: t=2.919780, itensity=3.031979, loss=0.427470
  5: t=3.328418, itensity=4.335602, loss=0.598241
  6: t=4.440399, itensity=4.643835, loss=0.922357
  7: t=4.511139, itensity=5.591233, loss=0.534293
Total loss: 3.912281
Average total loss over 5 runs: 2.034736 ± 0.968924


In [ ]:
import logging
import numpy as np
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math
import mpmath

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    grid_points: int = 500
    tolerance_scale: float = 1e-6
    # mpmath precision settings
    mp_dps: int = 50  # Decimal places for mpmath
    mp_eps: float = 1e-30  # Epsilon for mpmath comparisons

config = SimulationConfig()
mpmath.mp.dps = config.mp_dps  # Set global precision

# Create mpmath versions of constants for consistency
mp_zero = mpmath.mpf(0)
mp_eps = mpmath.mpf(config.mp_eps)
mp_one = mpmath.mpf(1)

def validate_parameters(model_params: ModelParams) -> bool:
    """Validate CIR parameters and check Feller condition"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"] 
        sigma = model_params["sigma"]
        
        assert kappa > 0, "kappa must be positive"
        assert theta > 0, "theta must be positive" 
        assert sigma > 0, "sigma must be positive"
        
        feller_condition = 2 * kappa * theta >= sigma**2
        if not feller_condition:
            logger.warning(f"Feller condition violated: 2kappa*theta={2*kappa*theta:.6f} < sigma^2={sigma**2:.6f}")
            
        logger.info(f"Parameters: κ={kappa:.6f}, θ={theta:.6f}, σ={sigma:.6f}")
        logger.info(f"Feller condition {'satisfied' if feller_condition else 'violated'}")
        
        return feller_condition
        
    except Exception as e:
        logger.error(f"Parameter validation failed: {e}")
        raise

def mp_hyp1f1(a: float, b: float, z: float) -> float:
    """Hypergeometric function evaluation with mpmath"""
    try:
        a_mp = mpmath.mpf(a)
        b_mp = mpmath.mpf(b)
        z_mp = mpmath.mpf(z)
        return float(mpmath.hyp1f1(a_mp, b_mp, z_mp))
    except Exception as e:
        logger.error(f"mp_hyp1f1 failed: {e}")
        return 1.0

def mp_hyp1f1_derivative(a: float, b: float, z: float, h: float = 1e-5) -> float:
    """Numerical derivative using mpmath"""
    try:
        a_mp = mpmath.mpf(a)
        b_mp = mpmath.mpf(b)
        z_mp = mpmath.mpf(z)
        h_mp = mpmath.mpf(h)
        
        f_plus = mpmath.hyp1f1(a_mp + h_mp, b_mp, z_mp)
        f_minus = mpmath.hyp1f1(a_mp - h_mp, b_mp, z_mp)
        derivative = (f_plus - f_minus) / (2 * h_mp)
        return float(derivative)
    except Exception as e:
        logger.error(f"mp_hyp1f1_derivative failed: {e}")
        return 0.0

def mp_laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform computation with mpmath"""
    try:
        kappa = mpmath.mpf(model_params["kappa"])
        theta = mpmath.mpf(model_params["theta"])
        sigma = mpmath.mpf(model_params["sigma"])
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        numerator = mpmath.hyp1f1(a, b, z1)
        denominator = mpmath.hyp1f1(a, b, z2)
        
        if abs(denominator) < mp_eps:
            logger.warning("Near-zero denominator in Laplace transform")
            return 0.99
            
        result = float(numerator / denominator)
        result = max(min(result, 0.999), 0.001)  # Clamp to valid probability range
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5

def mp_compute_coefficients(y: float, H: float, model_params: ModelParams, n_roots: int = 15) -> Tuple[List[float], List[float]]:
    """Compute eigenfunction expansion coefficients with mpmath"""
    start_time = time.time()
    
    try:
        kappa = mpmath.mpf(model_params["kappa"])
        theta = mpmath.mpf(model_params["theta"])
        sigma = mpmath.mpf(model_params["sigma"])
        
        c_bar = 2 * kappa * theta / sigma**2
        H_bar = 2 * kappa * H / sigma**2
        y_bar = 2 * kappa * y / sigma**2

        if c_bar < 1 or H_bar < c_bar:
            logger.error(f"Invalid parameters: c_bar={float(c_bar):.6f}, H_bar={float(H_bar):.6f}, y_bar={float(y_bar):.6f}")
            return [], []
        
        logger.debug(f"Computing coefficients: c_bar={float(c_bar):.6f}, H_bar={float(H_bar):.6f}, y_bar={float(y_bar):.6f}")
        
        def objective(alpha):
            return mpmath.hyp1f1(alpha, c_bar, H_bar)
        
        roots = []
        n = 0
        alpha_prev = mp_zero

        # Find the first root
        try:
            left = -c_bar / H_bar
            right = mp_zero
            alpha_curr = mpmath.findroot(objective, (left, right), solver='bisect')
            roots.append(alpha_curr)
        except Exception as e:
            logger.error(f"Failed to find first root: {e}")
            return [], []

        # Find subsequent roots
        while len(roots) < n_roots:
            n += 1
            alpha_hat = 2 * alpha_curr - alpha_prev
            
            # Expand search window dynamically
            left = alpha_hat - mpmath.mpf(2.0)
            right = alpha_hat + mpmath.mpf(0.1)
            found = False
            
            for attempt in range(10):
                try:
                    if objective(left) * objective(right) < 0:
                        alpha_next = mpmath.findroot(objective, (left, right), solver='bisect')
                        roots.append(alpha_next)
                        alpha_prev = alpha_curr
                        alpha_curr = alpha_next
                        found = True
                        break
                    else:
                        left -= mpmath.mpf(1.0)  # Expand left bound
                except:
                    left -= mpmath.mpf(1.0)
                    
            if not found:
                logger.warning(f"Could not bracket root {n+1}, stopping at {len(roots)} roots")
                break
        
        # Compute coefficients
        eta_n, beta_n = [], []
        for alpha in roots:
            try:
                alpha_f = float(alpha)
                if abs(alpha_f) < config.eps_zero:
                    logger.warning(f"Skipping near-zero alpha={alpha_f:.6f}")
                    eta_n.append(0.0)
                    beta_n.append(0.0)
                    continue
                
                eta = float(-kappa * alpha)
                eta_n.append(eta)
                
                # Compute derivative with mpmath
                derivative = mpmath.diff(lambda a: mpmath.hyp1f1(a, c_bar, H_bar), alpha)
                
                if abs(derivative) < mp_eps:
                    beta = 0.0
                else:
                    numerator = mpmath.hyp1f1(alpha, c_bar, y_bar)
                    beta = float(-numerator / (alpha * derivative))
                    
                beta_n.append(beta)
            except Exception as e:
                logger.warning(f"Coefficient computation failed: {e}")
                eta_n.append(0.0)
                beta_n.append(0.0)
        
        # Filter invalid coefficients
        valid_eta = []
        valid_beta = []
        for eta, beta in zip(eta_n, beta_n):
            if (np.isfinite(eta) and np.isfinite(beta) and 
                abs(eta) > config.eps_zero and abs(beta) > config.eps_zero):
                valid_eta.append(eta)
                valid_beta.append(beta)
        
        logger.debug(f"Computed {len(valid_eta)} valid coefficients in {time.time() - start_time:.3f}s")
        return valid_eta, valid_beta
        
    except Exception as e:
        logger.error(f"Coefficient computation failed: {e}")
        return [], []

def mp_compute_survival_probability_p1(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Compute survival probability with mpmath coefficients"""
    try:
        eta_n, beta_n = mp_compute_coefficients(y, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients available, using fallback")
            return 0.5
        
        p1 = 0.0
        for eta, beta in zip(eta_n, beta_n):
            if np.isfinite(beta) and np.isfinite(eta) and eta > config.eps_zero:
                p1 += beta * math.exp(-eta * tau)
        
        # Stability bounds
        p1 = max(min(p1, 0.999), 0.001)
        
        logger.debug(f"Survival probability: {p1:.6f}")
        return p1
        
    except Exception as e:
        logger.error(f"Survival probability computation failed: {e}")
        return 0.5

def mp_cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density computation with mpmath"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            return 0.0
            
        q = 2 * kappa * theta / sigma**2 - 1
        exp_term = math.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        # Compute in log space for stability
        log_factor = math.log(a) - (ax + b * y)
        log_power = (q / 2) * (math.log(ax) - math.log(by))
        
        # Bessel function with mpmath
        bessel_arg = 2 * math.sqrt(a * b * x * y)
        log_bessel = float(mpmath.log(mpmath.besseli(q, bessel_arg)))
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  # Prevent overflow
            return 0.0
            
        return math.exp(log_density)
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

def mp_select_dominating_intensity_H(y: float, model_params: ModelParams, 
                              t_C: float = 1.0, t_eta: float = 1.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H with mpmath"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = mp_laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, bounds=(H_min, H_max), method='bounded')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)
        H_ret = min(H_final, theta*8)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_ret
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)

def mp_sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density with mpmath"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        g_vals = np.array([mp_cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        eta_n, beta_n = mp_compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                mp_cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = mp_compute_survival_probability_p1(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)
        f_vals /= np.sum(f_vals)  # Normalize

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        
        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])

        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)

def mp_sample_hitting_time_from_v(nu_t: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample hitting time with mpmath"""
    try:
        eta_n, beta_n = mp_compute_coefficients(nu_t, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients for hitting time, using uniform")
            return np.random.uniform(0, tau)
        
        P = 1 - mp_compute_survival_probability_p1(nu_t, H, tau, model_params)
        P = max(P, config.eps_zero)
        
        U = np.random.uniform(0, P)
        
        def objective(s):
            integral = 0.0
            for beta, eta in zip(beta_n, eta_n):
                if eta > config.eps_zero:
                    integral += beta * (1 - math.exp(-eta * s))
            return integral - U

        try:
            result = brentq(objective, 0, tau)
            return min(result, tau)
        except:
            logger.warning("Hitting time root finding failed, using approximation")
            return np.random.uniform(0, tau * 0.8)
            
    except Exception as e:
        logger.error(f"Hitting time sampling failed: {e}")
        return np.random.uniform(0, tau * 0.5)

def sample_loss() -> float:
    """Sample loss from a uniform distribution"""
    return np.random.uniform(0.24, 0.96)

def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step using mpmath for precision
    """
    try:
        y = max(lambda_prev, config.eps_small)
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            H = mp_select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            tau = np.random.exponential(scale = 1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            p1 = mp_compute_survival_probability_p1(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            u1 = np.random.uniform()
            
            if u1 > p1:
                sigma_H = mp_sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                if t > T_max:
                    return None
                    
                y = H
                continue
            else:
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                if t > T_max:
                    return None
                
                nu_Tn = mp_sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function using mpmath for all precision-critical operations
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")
    
    if not validate_parameters(model_params):
        logger.warning("Proceeding despite Feller condition violation")
    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    gamma = model_params["gamma"]
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults


model_params = {
    "kappa": 2.62,
    "theta": 1.61,
    "sigma": 0.62,
    "gamma": 0.99
}

L_5 = []

for _ in range(10):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 10 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")

2025-06-04 09:27:00,153 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-04 09:27:00,153 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-06-04 09:27:00,153 - INFO - Feller condition satisfied
2025-06-04 09:27:00,155 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-06-04 09:27:00,537 - INFO - Computed survival probability p1=0.997124 for y=0.700000, H=2.295761, τ=0.405292
2025-06-04 09:27:00,537 - INFO - Process survives to t=0.405292, sampling conditional intensity
2025-06-04 09:28:04,950 - INFO - Default accepted at t=0.405292, ν=1.440000.
2025-06-04 09:28:04,953 - INFO - Iteration 1: Selected H=2.736571 for y=1.923359
2025-06-04 09:28:05,390 - INFO - Computed survival probability p1=0.975492 for y=1.923359, H=2.736571, τ=0.009763
2025-06-04 09:28:05,390 - INFO - Process survives to t=0.415055, sampling conditional intensity
2025-06-04 09:28:59,765 - INFO - Iteration 2: Selected H=2.699398 for y=1.860000
2025-06-04 09:29:00,203 - INFO - 

Generated 12 defaults:
  1: t=0.405292, itensity=1.923359, loss=0.488241
  2: t=0.487610, itensity=1.878981, loss=0.322203
  3: t=0.845085, itensity=1.628539, loss=0.352060
  4: t=2.493608, itensity=2.967287, loss=0.444282
  5: t=2.780636, itensity=4.208546, loss=0.894126
  6: t=3.204325, itensity=5.136992, loss=0.658074
  7: t=3.247790, itensity=5.332088, loss=0.508399
  8: t=4.174714, itensity=4.553852, loss=0.846965
  9: t=4.514764, itensity=3.787812, loss=0.422183
  10: t=4.800556, itensity=3.272563, loss=0.550847
  11: t=4.800680, itensity=4.036710, loss=0.705753
  12: t=4.962976, itensity=3.180872, loss=0.521863
Total loss: 6.714995


2025-06-04 09:43:37,522 - INFO - Computed survival probability p1=0.999000 for y=0.700000, H=2.295761, τ=0.090682
2025-06-04 09:43:37,522 - INFO - Process survives to t=0.090682, sampling conditional intensity
2025-06-04 09:44:35,612 - INFO - Iteration 2: Selected H=2.362843 for y=1.000000
2025-06-04 09:44:36,033 - INFO - Computed survival probability p1=0.750991 for y=1.000000, H=2.362843, τ=1.793995
2025-06-04 09:44:36,034 - INFO - Process survives to t=1.884677, sampling conditional intensity
2025-06-04 09:45:22,456 - INFO - Default accepted at t=1.884677, ν=1.820000.
2025-06-04 09:45:22,460 - INFO - Iteration 1: Selected H=2.979318 for y=2.279740
2025-06-04 09:45:22,910 - INFO - Computed survival probability p1=0.978109 for y=2.279740, H=2.979318, τ=0.048095
2025-06-04 09:45:22,910 - INFO - Process survives to t=1.932772, sampling conditional intensity
2025-06-04 09:46:19,085 - INFO - Default accepted at t=1.932772, ν=2.142955.
2025-06-04 09:46:19,089 - INFO - Iteration 1: Selected

Generated 11 defaults:
  1: t=1.884677, itensity=2.279740, loss=0.464383
  2: t=1.932772, itensity=2.955931, loss=0.821188
  3: t=2.669005, itensity=3.218725, loss=0.862606
  4: t=2.853545, itensity=3.729505, loss=0.514558
  5: t=2.902376, itensity=3.950017, loss=0.637128
  6: t=3.064352, itensity=3.181476, loss=0.380772
  7: t=3.076305, itensity=3.998601, loss=0.664698
  8: t=3.966535, itensity=1.740497, loss=0.323734
  9: t=4.160011, itensity=2.035033, loss=0.439427
  10: t=4.435881, itensity=1.908344, loss=0.344817
  11: t=4.685647, itensity=2.433821, loss=0.519012
Total loss: 5.972323


2025-06-04 10:13:49,161 - INFO - Computed survival probability p1=0.999000 for y=0.700000, H=2.295761, τ=0.184829
2025-06-04 10:13:49,162 - INFO - Process survives to t=0.184829, sampling conditional intensity
2025-06-04 10:14:49,565 - INFO - Iteration 2: Selected H=2.348069 for y=0.940000
2025-06-04 10:14:49,986 - INFO - Computed survival probability p1=0.999000 for y=0.940000, H=2.348069, τ=0.376876
2025-06-04 10:14:49,987 - INFO - Process survives to t=0.561705, sampling conditional intensity
2025-06-04 10:15:53,634 - INFO - Iteration 3: Selected H=2.477890 for y=1.380000
2025-06-04 10:15:54,065 - INFO - Computed survival probability p1=0.999000 for y=1.380000, H=2.477890, τ=0.083108
2025-06-04 10:15:54,066 - INFO - Process survives to t=0.644813, sampling conditional intensity
2025-06-04 10:16:51,565 - INFO - Default accepted at t=0.644813, ν=1.380000.
2025-06-04 10:16:51,569 - INFO - Iteration 1: Selected H=2.717269 for y=1.890871
2025-06-04 10:16:52,009 - INFO - Computed survival

In [ ]:
import logging
import numpy as np
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from scipy.special import hyp1f1 as scipy_hyp1f1
from scipy.special import iv as scipy_iv  # For modified Bessel function
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math
import mpmath

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    grid_points: int = 500
    tolerance_scale: float = 1e-6
    # mpmath precision settings
    mp_dps: int = 50  # Decimal places for mpmath
    mp_eps: float = 1e-30  # Epsilon for mpmath comparisons
    # Thresholds for switching to mpmath
    hyp1f1_threshold: float = 30.0  # Use mpmath for |z| > threshold
    bessel_threshold: float = 50.0  # Use mpmath for arg > threshold

config = SimulationConfig()

# Configure mpmath only when needed, not globally
def with_mp_precision(func):
    """Decorator to temporarily set mpmath precision for a function"""
    def wrapper(*args, **kwargs):
        old_dps = mpmath.mp.dps
        mpmath.mp.dps = config.mp_dps
        try:
            return func(*args, **kwargs)
        finally:
            mpmath.mp.dps = old_dps
    return wrapper

def hyp1f1(a: float, b: float, z: float) -> float:
    """Hypergeometric function with automatic method selection"""
    try:
        # Use scipy for moderate values (faster)
        if abs(z) < config.hyp1f1_threshold and abs(a) < 20 and abs(b) < 100:
            result = scipy_hyp1f1(a, b, z)
            if np.isfinite(result) and abs(result) < 1e100:
                return result
                
        # Fall back to mpmath for numerically challenging cases
        with mpmath.workdps(config.mp_dps):
            a_mp = mpmath.mpf(a)
            b_mp = mpmath.mpf(b)
            z_mp = mpmath.mpf(z)
            return float(mpmath.hyp1f1(a_mp, b_mp, z_mp))
    except Exception as e:
        logger.warning(f"hyp1f1 failed: {e}, using fallback")
        return 1.0

def hyp1f1_derivative(a: float, b: float, z: float) -> float:
    """Compute derivative of hypergeometric function efficiently"""
    try:
        # Use finite difference for moderate values
        if abs(z) < config.hyp1f1_threshold and abs(a) < 20:
            h = 1e-6
            f_plus = hyp1f1(a + h, b, z)
            f_minus = hyp1f1(a - h, b, z)
            return (f_plus - f_minus) / (2 * h)
        
        # Use mpmath for higher precision when needed
        with mpmath.workdps(config.mp_dps):
            a_mp = mpmath.mpf(a)
            b_mp = mpmath.mpf(b)
            z_mp = mpmath.mpf(z)
            return float(mpmath.diff(lambda x: mpmath.hyp1f1(x, b_mp, z_mp), a_mp))
    except Exception as e:
        logger.warning(f"hyp1f1_derivative failed: {e}")
        return 0.0

def laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform with selective mpmath usage"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        # Use mpmath only for challenging cases
        use_mpmath = (abs(z1) > config.hyp1f1_threshold or 
                     abs(z2) > config.hyp1f1_threshold or
                     abs(a) > 20)
        
        if use_mpmath:
            with mpmath.workdps(config.mp_dps):
                numerator = mpmath.hyp1f1(mpmath.mpf(a), mpmath.mpf(b), mpmath.mpf(z1))
                denominator = mpmath.hyp1f1(mpmath.mpf(a), mpmath.mpf(b), mpmath.mpf(z2))
                result = float(numerator / denominator)
        else:
            numerator = hyp1f1(a, b, z1)
            denominator = hyp1f1(a, b, z2)
            
            if abs(denominator) < config.eps_zero:
                logger.warning(f"Near-zero denominator in Laplace transform: a={a:.6f}, b={b:.6f}, z2={z2:.6f}")
                return 0.99
                
            result = numerator / denominator
            
        # Stability bounds
        result = max(min(result, 0.999), 0.001)
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5

def compute_coefficients(y: float, H: float, model_params: ModelParams, n_roots: int = 15) -> Tuple[List[float], List[float]]:
    """Compute eigenfunction expansion coefficients with optimized precision"""
    start_time = time.time()
    
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        c_bar = 2 * kappa * theta / sigma**2
        H_bar = 2 * kappa * H / sigma**2
        y_bar = 2 * kappa * y / sigma**2

        if c_bar < 1 or H_bar < c_bar:
            logger.error(f"Invalid parameters: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
            return [], []
        
        logger.debug(f"Computing coefficients: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
        
        # Root finding needs mpmath precision
        with mpmath.workdps(config.mp_dps):
            mp_kappa = mpmath.mpf(kappa)
            mp_c_bar = mpmath.mpf(c_bar)
            mp_H_bar = mpmath.mpf(H_bar)
            mp_y_bar = mpmath.mpf(y_bar)
            
            def objective(alpha):
                return mpmath.hyp1f1(alpha, mp_c_bar, mp_H_bar)
            
            roots = []
            n = 0
            alpha_prev = mpmath.mpf(0)

            # Find the first root
            try:
                left = -mp_c_bar / mp_H_bar
                right = mpmath.mpf(0)
                alpha_curr = mpmath.findroot(objective, (left, right), solver='bisect')
                roots.append(alpha_curr)
            except Exception as e:
                logger.error(f"Failed to find first root: {e}")
                return [], []

            # Find subsequent roots
            while len(roots) < n_roots:
                n += 1
                alpha_hat = 2 * alpha_curr - alpha_prev
                
                # Expand search window dynamically
                left = alpha_hat - mpmath.mpf(2.0)
                right = alpha_hat + mpmath.mpf(0.1)
                found = False
                
                for attempt in range(10):
                    try:
                        if objective(left) * objective(right) < 0:
                            alpha_next = mpmath.findroot(objective, (left, right), solver='bisect')
                            roots.append(alpha_next)
                            alpha_prev = alpha_curr
                            alpha_curr = alpha_next
                            found = True
                            break
                        else:
                            left -= mpmath.mpf(1.0)  # Expand left bound
                    except:
                        left -= mpmath.mpf(1.0)
                        
                if not found:
                    logger.warning(f"Could not bracket root {n+1}, stopping at {len(roots)} roots")
                    break
            
            # Compute coefficients - still using mpmath for stability
            eta_n, beta_n = [], []
            for alpha in roots:
                try:
                    alpha_f = float(alpha)
                    if abs(alpha_f) < config.eps_zero:
                        logger.warning(f"Skipping near-zero alpha={alpha_f:.6f}")
                        eta_n.append(0.0)
                        beta_n.append(0.0)
                        continue
                    
                    eta = float(-mp_kappa * alpha)
                    eta_n.append(eta)
                    
                    # Compute derivative
                    derivative = mpmath.diff(lambda a: mpmath.hyp1f1(a, mp_c_bar, mp_H_bar), alpha)
                    
                    if abs(derivative) < config.mp_eps:
                        beta = 0.0
                    else:
                        numerator = mpmath.hyp1f1(alpha, mp_c_bar, mp_y_bar)
                        beta = float(-numerator / (alpha * derivative))
                        
                    beta_n.append(beta)
                except Exception as e:
                    logger.warning(f"Coefficient computation failed: {e}")
                    eta_n.append(0.0)
                    beta_n.append(0.0)
        
        # Filter invalid coefficients (use standard float operations)
        valid_eta = []
        valid_beta = []
        for eta, beta in zip(eta_n, beta_n):
            if (np.isfinite(eta) and np.isfinite(beta) and 
                abs(eta) > config.eps_zero and abs(beta) > config.eps_zero):
                valid_eta.append(eta)
                valid_beta.append(beta)
        
        logger.debug(f"Computed {len(valid_eta)} valid coefficients in {time.time() - start_time:.3f}s")
        return valid_eta, valid_beta
        
    except Exception as e:
        logger.error(f"Coefficient computation failed: {e}")
        return [], []

def compute_survival_probability_p1(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Compute survival probability with standard floating point math when possible"""
    try:
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients available, using fallback")
            return 0.5
        
        p1 = 0.0
        # Use standard math for summation - faster than mpmath
        for eta, beta in zip(eta_n, beta_n):
            if np.isfinite(beta) and np.isfinite(eta) and eta > config.eps_zero:
                p1 += beta * math.exp(-eta * tau)
        
        # Stability bounds
        p1 = max(min(p1, 0.999), 0.001)
        logger.debug(f"Survival probability: {p1:.6f}")
        return p1
        
    except Exception as e:
        logger.error(f"Survival probability computation failed: {e}")
        return 0.5

def cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density with selective mpmath for Bessel function only"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            return 0.0
            
        q = 2 * kappa * theta / sigma**2 - 1
        exp_term = math.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        # Compute in log space for stability
        log_factor = math.log(a) - (ax + by)
        log_power = (q / 2) * (math.log(ax) - math.log(by))
        
        # Bessel function - use scipy for moderate values, mpmath for large ones
        bessel_arg = 2 * math.sqrt(a * b * x * y)
        
        if bessel_arg < config.bessel_threshold and q < 100:
            try:
                # Use scipy's iv (faster)
                bessel_value = scipy_iv(q, bessel_arg)
                log_bessel = np.log(bessel_value) if bessel_value > 0 else -np.inf
            except:
                # Fall back to mpmath
                log_bessel = float(mpmath.log(mpmath.besseli(q, bessel_arg)))
        else:
            # Use mpmath for large arguments
            log_bessel = float(mpmath.log(mpmath.besseli(q, bessel_arg)))
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  # Prevent overflow
            return 0.0
            
        return math.exp(log_density)
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

def select_dominating_intensity_H(y: float, model_params: ModelParams, 
                         t_C: float = 1.0, t_eta: float = 1.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H - no need for mpmath here"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, bounds=(H_min, H_max), method='bounded')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)
        H_ret = min(H_final, theta*8)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_ret
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)

def sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density - no need for mpmath in sampling"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        g_vals = np.array([cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = compute_survival_probability_p1(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)
        f_vals /= np.sum(f_vals)  # Normalize

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        
        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])

        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)

def sample_hitting_time_from_v(nu_t: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample hitting time - no need for mpmath in integration"""
    try:
        eta_n, beta_n = compute_coefficients(nu_t, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients for hitting time, using uniform")
            return np.random.uniform(0, tau)
        
        P = 1 - compute_survival_probability_p1(nu_t, H, tau, model_params)
        P = max(P, config.eps_zero)
        
        U = np.random.uniform(0, P)
        
        def objective(s):
            integral = 0.0
            for beta, eta in zip(beta_n, eta_n):
                if eta > config.eps_zero:
                    integral += beta * (1 - math.exp(-eta * s))
            return integral - U

        try:
            result = brentq(objective, 0, tau)
            return min(result, tau)
        except:
            logger.warning("Hitting time root finding failed, using approximation")
            return np.random.uniform(0, tau * 0.8)
            
    except Exception as e:
        logger.error(f"Hitting time sampling failed: {e}")
        return np.random.uniform(0, tau * 0.5)

def sample_loss() -> float:
    """Sample loss from a uniform distribution"""
    return np.random.uniform(0.24, 0.96)

def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step using standard floating point
    """
    try:
        y = max(lambda_prev, config.eps_small)
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            H = select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            tau = np.random.exponential(scale = 1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            p1 = compute_survival_probability_p1(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            u1 = np.random.uniform()
            
            if u1 > p1:
                sigma_H = sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                if t > T_max:
                    return None
                    
                y = H
                continue
            else:
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                if t > T_max:
                    return None
                
                nu_Tn = sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function using mpmath for all precision-critical operations
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")

    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    gamma = model_params["gamma"]
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults


model_params = {
    "kappa": 2.62,
    "theta": 1.61,
    "sigma": 0.62,
    "gamma": 2.99  # Original paper value
}

L_5 = []

for _ in range(2):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 2 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")

2025-06-04 10:21:13,476 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-04 10:21:13,479 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-06-04 10:21:13,860 - INFO - Computed survival probability p1=0.777663 for y=0.700000, H=2.295761, τ=1.502569
2025-06-04 10:21:14,630 - INFO - Iteration 2: Selected H=2.991425 for y=2.295761
2025-06-04 10:21:15,079 - INFO - Computed survival probability p1=0.972464 for y=2.295761, H=2.991425, τ=0.188799
2025-06-04 10:21:15,080 - INFO - Process survives to t=1.339577, sampling conditional intensity
2025-06-04 10:21:59,349 - INFO - Iteration 3: Selected H=2.812076 for y=2.043228
2025-06-04 10:21:59,806 - INFO - Computed survival probability p1=0.970246 for y=2.043228, H=2.812076, τ=0.203402
2025-06-04 10:21:59,806 - INFO - Process survives to t=1.542979, sampling conditional intensity
2025-06-04 10:22:44,159 - INFO - Default accepted at t=1.542979, ν=2.124957.
2025-06-04 10:22:44,168 - INFO - Iteration 1: Selected H=4.